In [1]:
import pandas as pd
from config import DATA_DOWNLOAD_URL
import os

In [2]:
def get_market_year(row): #make a market year row in datas
    year = row['date'].year
    month = row['date'].month
    if month >= 9:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [3]:
hist = pd.read_csv(f"{DATA_DOWNLOAD_URL}/ERA5/1940-2025_vpd.csv").drop("Unnamed: 0", axis=1)

In [4]:
hist["date"] = pd.to_datetime(hist["date"])
hist = hist[hist["date"] >= "2019-09-01"] #year to start, i will take arbitrary last 5 market years
hist["MY"] = hist.apply(get_market_year, axis=1)
hist["MONTH"] = hist["date"].dt.month
hist = hist[~hist["dep"].isin([75,92,93])] #remove 3 deps cause we have no data
current = hist[hist["MY"] == "2024/2025"].copy() #copy current MY
hist = hist[hist["MY"] != "2024/2025"] #remove current MY

In [ ]:
meanHist = hist[["dep", "MONTH", "vpd_min", "vpd_mean", "vpd_max"]].groupby(["dep", "MONTH"]).mean().reset_index()

In [5]:
meanCurrent = current[["dep", "MONTH", "vpd_min", "vpd_mean", "vpd_max"]].groupby(["dep", "MONTH"]).mean().reset_index()

In [ ]:
meanHist.to_csv("../YieldModel/data/forecast/currentYear/vpd_mean_historical.csv")

In [7]:
meanCurrent.to_csv("../YieldModel/data/forecast/currentYear/vpd_mean_current.csv")